In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [15]:
import pandas as pd
import numpy as np
import os
import json
import re
import os
import nltk
from functools import reduce
from itertools import chain
import pickle
import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

from nltk.corpus import stopwords 
nltk.download('stopwords')
stops = set(stopwords.words('english'))
from nltk.tokenize import sent_tokenize, word_tokenize, string_span_tokenize
nltk.download('punkt')
stops.add("!")
import csv

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# The file directories
f = '/content/drive/My Drive/Hallo_Data/clean_data/'
folder2 = '/content/drive/My Drive/Hallo_Data/winter_project/models/'
folder = '/content/drive/My Drive/Hallo_Data/messages_with_text/'
full = folder+"full_messages.pkl"
full1 = folder+"full_messages1.pkl"
full2 = folder+"full_messages2.pkl"
message = "Welcome to Hallo\n            \nThe best way to be fluent is by actually speaking and practicing everyday, and Hallo provides the opportunity to speak English for free.\n            \nLet's make some new friends and practice together!\n            \nOne friendship away from fluency,\n            \nThe Hallo team"

## Naive Bayes for message spam/banned detection

In [0]:
def load_clean_data(kind="banned", full=False):

    if os.path.exists(folder+kind+"_messages.pkl"):
        temp =  pd.read_pickle(folder+kind+"_messages.pkl")
        if full:
            return temp
    else:
    
        b = pd.read_pickle(f+"bans.pkl")
        m = pd.read_csv(folder+"messages_text_0.csv")
        if kind=="banned":
            temp = m[m.id.isin(b.user_id)]
        else:
            temp = m[~m.id.isin(b.user_id)]

        for i in range(1,10):
            i = str(i)
            m = pd.read_csv(folder+"messages_text_"+i+".csv")
            if kind=="banned":
                m1 = m[m.id.isin(b.user_id)]
            else:
                m1 = m[~m.id.isin(b.user_id)]
            temp = pd.concat((temp, m1))

        temp = temp[temp.text != message]
        temp.to_pickle(folder+kind+"_messages.pkl")

        if full:
            return temp

    temp.drop(columns=["roomId"], inplace=True)
    temp['label'] = 1 if kind == "banned" else 0
    pd.to_pickle(temp[['text', 'label']], folder+kind+"spam_prepared.pkl")

    return temp

In [0]:
banned = load_clean_data()
not_banned = load_clean_data(kind="not_banned")

## Baseline

In [0]:
def baseline(n=1, ban=True):

    df = banned if ban else not_banned.iloc[:len(banned),:]

    model, vectorizer, test_X, test_y = fit_naive_bayes()
    scores = []

    for id in df.id.unique():
        test_counts = vectorizer.transform(df[df.id == id].text.values.astype(str))

        preds = 0
        total = 0

        for cnt in test_counts:
            pred = model.predict(cnt)
            total += 1
            preds += pred

        score = preds/total
        # print(score[0],",")
        scores.append(score)

    print(np.mean(scores))
    print(np.median(scores))
    print("Q1 quantile of arr : ", np.quantile(scores, .25)) 
    print("Q2 quantile of arr : ", np.quantile(scores, .50)) 
    print("Q3 quantile of arr : ", np.quantile(scores, .75)) 
    print("100th quantile of arr : ", np.quantile(scores, .1))



In [0]:
baseline()

spliting
For 150000 messages
train len 105000
test len 45000
data setting
Counting
0.47440208015097207
0.44114396242055814
Q1 quantile of arr :  0.2669112886738449
Q2 quantile of arr :  0.44114396242055814
Q3 quantile of arr :  0.6687197568747788
100th quantile of arr :  0.1528846153846154


In [0]:
baseline(ban=False)

spliting
For 150000 messages
train len 105000
test len 45000
data setting
Counting
0.19624873329991632
0.1
Q1 quantile of arr :  0.0
Q2 quantile of arr :  0.1
Q3 quantile of arr :  0.2777777777777778
100th quantile of arr :  0.0


## Message classification

In [0]:
def fit_naive_bayes(n=150000, prnt=False):
    print("spliting")
    print("For",n,"messages")

    b_data = banned.sample(n)
    nb_data = not_banned.sample(n)

    train_X_ban, test_X_ban, train_y_ban, test_y_ban = train_test_split(b_data.drop(columns=["label"]), b_data.label,test_size=.3)
    train_X_nban, test_X_nban, train_y_nban, test_y_nban = train_test_split(nb_data.drop(columns=["label"]), nb_data.label,test_size=.3)

    print("train len", len(train_X_ban))
    print("test len", len(test_X_ban))
    print("data setting")
    train_X = pd.concat((train_X_ban,train_X_nban))
    train_y = pd.concat((train_y_ban,train_y_nban))
    test_X = pd.concat((test_X_ban,test_X_nban))
    test_y = pd.concat((test_y_ban,test_y_nban))

    print("Counting")
    vectorizer = CountVectorizer()
    counts = vectorizer.fit_transform(train_X.text.values.astype(str))

    params = {'alpha': np.linspace(0,2,20),
              "fit_prior": [True, False]}

    grid_clf = GridSearchCV(MultinomialNB(), param_grid=params)
    grid_clf.fit(counts, train_y.values)
    best_model = grid_clf.best_estimator_

    pickle.dump(best_model, open(folder2+"message_spam_model.pkl", 'wb'))

    if prnt:
        print("Scoring")
        test_counts = vectorizer.transform(test_X.text.values.astype(str))
        print("Score:", best_model.score(test_counts, test_y.values))

        print("\n Testing random inputs: \n")
        for _ in range(5):
            i = np.random.randint(len(test_X))
            print("Text:",test_X.text.values[i])
            print("Predicted Spam:",best_model.predict(test_counts[i]))
            print("Actually spam:", test_y.values[i])
            print()

    return best_model, vectorizer, test_X, test_y


In [0]:
fit_naive_bayes(prnt=True)

spliting
For 150000 messages
train len 105000
test len 45000
data setting
Counting
Scoring
Score: 0.6225555555555555

 Testing random inputs: 

Text: But actually I think being  a teacher can also get many interesting stories from students and coworkers
Predicted Spam: [0]
Actually spam: 0

Text: Can I have a Picture of you now
Predicted Spam: [1]
Actually spam: 0

Text: Call me
Predicted Spam: [1]
Actually spam: 0

Text: Hope you understand
Predicted Spam: [0]
Actually spam: 1

Text: Hello
Predicted Spam: [1]
Actually spam: 1



(MultinomialNB(alpha=0.10526315789473684, class_prior=None, fit_prior=True),
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=1.0, max_features=None, min_df=1,
                 ngram_range=(1, 1), preprocessor=None, stop_words=None,
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None),
                                                       text  ...                            id
 4455193                                                 Hi  ...  yyaOhFcQuhVsumjMYS913XVvuNb2
 238988                                                Good  ...  CAUoUpvOwrQOiLxzvP21JTF53Q32
 3549562                                               Baby  ...  Xs5ewEpHsRYYRyTEzCmNK8qIaav2
 72101    He regret and say sorry and promise not to do ...  ...  1K4DEc3HjBhjqq2TqSP5eDdmpMp1
 3673749  But I never clos

spliting
For 150000 messages
train len 105000
test len 45000
data setting
Counting
Scoring
Score: 0.6224222222222222

 Testing random inputs: 

Text: U too 💕
Predicted Spam: [0]
Actually spam: 1

Text: How
Predicted Spam: [0]
Actually spam: 0

Text: You are beautiful
Predicted Spam: [0]
Actually spam: 0

Text: Do you understand
Predicted Spam: [0]
Actually spam: 0

Text: What happened
Predicted Spam: [0]
Actually spam: 1



(MultinomialNB(alpha=0.21052631578947367, class_prior=None, fit_prior=True),
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=1.0, max_features=None, min_df=1,
                 ngram_range=(1, 1), preprocessor=None, stop_words=None,
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None),
                                                       text  ...                            id
 458731   I remembered  that one day I shared gp rule he...  ...  uMYyHEJ0RnVQldi4JxlR1nUjXF73
 2396196                                              inbok  ...  7nNOPBjrxLR5pDRaIk3XlkMd2VP2
 2698153                          she is in thr other group  ...  copjsSmj3aZyUfoGpv9TMfQ5Zsq1
 3756940  Silva you truly lifted me up thank you so much...  ...  CwWT4iJeI6XvTaZay1wawpr71iz2
 2318602            Gue ka

## Bad Actor Identification

In [0]:
def evaluate_model(true_y, pred_y):
    tn, fp, fn, tp = confusion_matrix(true_y, pred_y).ravel()
    print("TN:", tn, " FP:",fp, " FN:",fn, " TP:", tp)
    print("TPR: ",tp / (tp + fn))
    print("PPV: ", tp/(tp + fp))
    print("TNR: ", tn/(tn + fn))

In [0]:
def predict_banned():

    model, vectorizer, test_X, test_y = fit_naive_bayes()

    bans = test_X[test_y == 1].id
    not_bans = test_X[test_y == 0].id

    y_preds = []
    y_truth = []

    for data in [(not_bans,"not_bans"),(bans,"bans")]:
        d = data[0]
        correct = 0
        
        for i, userid in enumerate(pd.unique(d)):
            df = test_X[test_X['id'].str.contains(userid)]
            test_counts = vectorizer.transform(df.text.values.astype(str))

            preds = 0
            total = 0

            for cnt in test_counts:
                pred = model.predict(cnt)
                total += 1
                preds += pred

            score = preds/total

            if score >= .25:
                y_preds.append(1)
            else:
                y_preds.append(0)

            if data[1]=="bans":
                y_truth.append(1)
            else:
                y_truth.append(0)
                    
    print(classification_report(y_truth, y_preds))
    evaluate_model(y_truth, y_preds)

    return 

In [26]:
predict_banned()

spliting
For 150000 messages
train len 105000
test len 45000
data setting
Counting
              precision    recall  f1-score   support

           0       0.98      0.72      0.83     17212
           1       0.15      0.74      0.25      1155

    accuracy                           0.72     18367
   macro avg       0.56      0.73      0.54     18367
weighted avg       0.92      0.72      0.79     18367

TN: 12416  FP: 4796  FN: 301  TP: 854
TPR:  0.7393939393939394
PPV:  0.1511504424778761
TNR:  0.9763308956514901
